# TODO

# LinearRegression from Scratch


다음 가상 데이터를 이용해 사과와 오렌지 수확량을 예측하는 선형회귀 모델을 정의한다. 


|온도(F)|강수량(mm)|습도(%)|사과생산량(ton)|오렌지생산량|
|-|-|-|-:|-:|
|73|67|43|56|70|
|91|88|64|81|101|
|87|134|58|119|133|
|102|43|37|22|37|
|69|96|70|103|119|

```
사과수확량  = w11 * 온도 + w12 * 강수량 + w13 * 습도 + b1
오렌지수확량 = w21 * 온도 + w22 * 강수량 + w23 *습도 + b2
```

- `온도`, `강수량`, `습도` 값이 **사과**와, **오렌지 수확량**에 어느정도 영향을 주는지 가중치를 찾는다.
    - 모델은 사과의 수확량, 오렌지의 수확량 **두개의 예측결과를 출력**해야 한다.
    - 사과에 대해 예측하기 위한 weight 3개와 오렌지에 대해 예측하기 위한 weight 3개 이렇게 두 묶음, 총 6개의 weight를 정의하고 학습을 통해 가장 적당한 값을 찾는다.

## step 1.
## Train Dataset
- Train data는 feature(input)와 target(output) 각각 2개의 행렬로 구성한다.
- Feature는 `온도, 강수량, 습도` 3개의 변수를 가진다.
- Target은 `사과수확량, 오렌지 수확량` 2개의 값이다.

In [ ]:
# input/output 2차원리스트 생성하기기
# input: 생산환경 (temp, rainfall, humidity) - (5, 3)
environs = []

# output: 생산량 - (apples, oranges) - (5, 2)
apple_orange_output = []

In [ ]:
import torch
# Dataset을 torch.Tensor(dtype은 float32)로 생성 후 X,y의 shape 확인하기

# 1. Tensoer 생성
X = 
y = 

# 2. shape 확인

## step 2. weight/bias 정의하기

In [ ]:
# random(torch.randn) 값을 이용해서 weight/bias 를 정의하기하고 size 확인인

# 1. weight, bias 정의(requires_grad=True)
weight =
bias = 

# 2. weight, bias size 확인


## step3. 모델링

In [ ]:
# 모델은 y = X * w + b 꼴, loss함수는 MSE로 정의

## 1. 모델 정의
def model(X):
    return

## 2. loss 함수(MSE)
def loss_fn(pred, y):
    return

# 3. epochs =5000, lr = 0.00001로 모델링 완성하기기


for in :

    # 1. 추론
    
    
    # 2. loss 계산
    

    # 3. 파라미터 들의 gradient 계산
    

    # 4. 파라미터 업데이트
    

    # 5. gradient 초기화
    

    ## 100 epoch, 마지막 epoch에서 loss의 log를 출력
    if epoch % 100 == 0 or epoch == epochs-1:
        print(f"[{epoch+1:04d}/{epochs}] - {loss.item():.5f}")

## step 4. 기존의 X와 new_x로 model 결과확인하기

In [ ]:
# 1. 위 모델에 X로 다시 추론하여, 예측값 p와 정답값 y 비교하기

p =

print(f"예측값(p):\n", p)
print(f"정답값(y):\n", y)

# 2. 새로운 데이터 [68, 82, 56] 넣어 결과 확인(dtype은 float32 & .unsqueeze(dim=0))

new_x =

model(new_x)